# Eat Safe, Love

## Notebook Set Up

In [2]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our database
print(db.list_collection_names())

['artifacts', 'establishments']


In [6]:
# assign the collection to a variable
establishments = db['establishments']
artifacts = db['artifacts']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [7]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
print(f"There are {artifacts.count_documents(query)} documents with a 'Hygiene' score of 20.")

# Display the first document in the results using pprint
results = artifacts.find(query)
print("")
print("The first document is:")
print("-"*50)
if artifacts.count_documents(query) > 0:
	pprint(artifacts[0])
else:
	print("No documents found.")

There are 41 documents with a 'Hygiene' score of 20.

The first document is:
--------------------------------------------------
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food'), 'artifacts.0')


In [8]:
import os

# Create the directory if it does not exist
os.makedirs("csv_outputs", exist_ok=True)

# Convert the result to a Pandas DataFrame
hygiene_df = pd.json_normalize(results)

# Export DataFrame to csv file
hygiene_df.to_csv("csv_outputs/hygiene_df.csv", index=False, header=True)

# Display the number of rows in the DataFrame
print(f"The number of rows in this DataFrame is: {len(hygiene_df)}")
print("")

# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

The number of rows in this DataFrame is: 41



,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,geocode.longitude,geocode.latitude,meta.dataSource,meta.extractDate,meta.itemCount,meta.returncode,meta.totalCount,meta.totalPages,meta.pageSize,meta.pageNumber
0,673ff4fe1a87a14d8971b59a,110681,0,4029,The Chase Rest Home,Caring Premises,5,5-6 Southfields Road,Eastbourne,East Sussex,...,0.27694,50.769705,None,0001-01-01T00:00:00,0,None,0,0,0,0
1,673ff4fe1a87a14d8971b916,612039,0,1970/FOOD,Brenalwood,Caring Premises,5,Hall Lane,Walton-on-the-Naze,Essex,...,1.278721,51.857536,None,0001-01-01T00:00:00,0,None,0,0,0,0
2,673ff4fe1a87a14d8971bc25,730933,0,1698/FOOD,Melrose Hotel,Hotel/bed & breakfast/guest house,7842,53 Marine Parade East,Clacton On Sea,Essex,...,1.15927,51.789429,None,0001-01-01T00:00:00,0,None,0,0,0,0
3,673ff4fe1a87a14d8971be15,172735,0,PI/000023858,Seaford Pizza,Takeaway/sandwich shop,7844,4 High Street,Seaford,East Sussex,...,0.10202,50.770885,None,0001-01-01T00:00:00,0,None,0,0,0,0
4,673ff4fe1a87a14d8971be1f,172953,0,PI/000024532,Golden Palace,Restaurant/Cafe/Canteen,1,5 South Street,Seaford,East Sussex,...,0.101446,50.770724,None,0001-01-01T00:00:00,0,None,0,0,0,0
5,673ff4fe1a87a14d8971c7ce,512854,0,12/00816/BUTH,Ashby's Butchers,Retailers - other,4613,777 Southchurch Road,Southend-On-Sea,Essex,...,0.736349,51.541448,None,0001-01-01T00:00:00,0,None,0,0,0,0
6,673ff4fe1a87a14d8971c9dd,1537089,0,22/00224/RESTUN,South Sea Express Cuisine,Restaurant/Cafe/Canteen,1,33 Alexandra Street,Southend-on-sea,Essex,...,0.7121671,51.5350065,None,0001-01-01T00:00:00,0,None,0,0,0,0
7,673ff4ff1a87a14d8971df08,155648,0,EH/00006058,Golden Palace,Takeaway/sandwich shop,7844,7 London Road,Rayleigh,Essex,...,0.602364,51.591515,None,0001-01-01T00:00:00,0,None,0,0,0,0
8,673ff4ff1a87a14d8971e350,1012883,0,17/00110/MIXED/S,The Tulip Tree,Restaurant/Cafe/Canteen,1,3 The Village,Chiddingstone,KENT,...,0.146449998021126,51.1859474182129,None,0001-01-01T00:00:00,0,None,0,0,0,0
9,673ff4ff1a87a14d8971eb66,644109,0,41811,F & S,Retailers - other,4613,,81 Southernhay,Basildon,...,0.462307,51.57005,None,0001-01-01T00:00:00,0,None,0,0,0,0


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [17]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
# query = { "LocalAuthorityName": { "$regex": "London", "$options": "i" }, "RatingValue": { "$gte": 4 } }


query = {
    "LocalAuthorityName": {"$regex": "London", "$options": "i"},
    "$expr": {
        "$gte": [
            {
                "$convert": {
                    "input": "$RatingValue",
                    "to": "int",
                    "onError": 0,  # If conversion fails, default to 0
                    "onNull": 0    # If RatingValue is null, default to 0
                }
            },
            4
        ]
    }
}

# Use count_documents to display the number of documents in the result
print(f"There are {artifacts.count_documents(query)} documents for artifacts within 'London' with a 'RatingValue' >= 4.")

# Display the first document in the results using pprint
results = artifacts.find(query)
print("")
print("The first document is:")
print("-"*50)
first_result = results.next() if artifacts.count_documents(query) > 0 else None
if first_result:
	pprint(first_result)
else:
	print("No documents found.")

There are 33 documents for artifacts within 'London' with a 'RatingValue' >= 4.

The first document is:
--------------------------------------------------
{'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4627.439467780196,
 'FHRSID': 621707,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'LocalAuthorityCode': '508',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'ME12',
 'RatingDate': '2021-10-18T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('673ff4fe1a87a14d8971cfb4'),
 'geoco

In [18]:
from pprint import pprint

# Define the query using $convert for safer conversion from string to int
query = {
    "LocalAuthorityName": {"$regex": "London", "$options": "i"},
    "$expr": {
        "$gte": [
            {
                "$convert": {
                    "input": "$RatingValue",
                    "to": "int",
                    "onError": 0,  # If conversion fails, default to 0
                    "onNull": 0    # If RatingValue is null, default to 0
                }
            },
            4
        ]
    }
}

# Count the documents that match the query
document_count = artifacts.count_documents(query)
print(f"There are {document_count} documents for artifacts within 'London' with a 'RatingValue' >= 4.")

# Retrieve and display the first matching document
results = artifacts.find(query)
print("\nThe first document is:")
print("-" * 50)
if document_count > 0:
    first_result = results.next()
    pprint(first_result)
else:
    print("No documents found.")


There are 33 documents for artifacts within 'London' with a 'RatingValue' >= 4.

The first document is:
--------------------------------------------------
{'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4627.439467780196,
 'FHRSID': 621707,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'LocalAuthorityCode': '508',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'ME12',
 'RatingDate': '2021-10-18T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('673ff4fe1a87a14d8971cfb4'),
 'geoco

In [19]:
# Convert the result to a Pandas DataFrame
london_rating = pd.json_normalize(results)

# Export DataFrame to csv file
london_rating.to_csv("csv_outputs/london_rating.csv", index=False, header=True)

# Display the number of rows in the DataFrame
print(f"The number of rows in this DataFrame is: {len(london_rating)}")
print("")

# Display the first 10 rows of the DataFrame
london_rating.head(10)

The number of rows in this DataFrame is: 32



,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,geocode.longitude,geocode.latitude,meta.dataSource,meta.extractDate,meta.itemCount,meta.returncode,meta.totalCount,meta.totalPages,meta.pageSize,meta.pageNumber
0,673ff4fe1a87a14d8971d2da,1130836,0,PI/000034075,Mv City Cruises Erasmus,Other catering premises,7841,Cherry Garden Pier,Cherry Garden Street Rotherhithe,Charlton,...,0.249255999922752,51.1236228942871,None,0001-01-01T00:00:00,0,None,0,0,0,0
1,673ff4ff1a87a14d8971de24,293783,0,PI/000002614,Benfleet Motor Yacht Club,Other catering premises,7841,Ferry Road,South Benfleet Essex,,...,0.561954,51.543831,None,0001-01-01T00:00:00,0,None,0,0,0,0
2,673ff4ff1a87a14d8971ec23,1315095,0,PI/000036464,Coombs Catering t/a The Lock and Key,Restaurant/Cafe/Canteen,1,Leslie Ford House,Tilbury,Charlton,...,0.346336990594864,51.4640655517578,None,0001-01-01T00:00:00,0,None,0,0,0,0
3,673ff4ff1a87a14d8971ec25,294474,0,PI/000014647,Tilbury Seafarers Centre,Restaurant/Cafe/Canteen,1,Tenants Row Tilbury Docks,Tilbury Essex,,...,0.346325,51.464078,None,0001-01-01T00:00:00,0,None,0,0,0,0
4,673ff4ff1a87a14d8971f344,294900,0,PI/000019066,Mv Valulla,Other catering premises,7841,Reeds River Cruises LtdKings ReachRiver Thames...,London,,...,0.287555,51.504071,None,0001-01-01T00:00:00,0,None,0,0,0,0
5,673ff4ff1a87a14d8972143a,293756,0,PI/000002538,Tereza Joanne,Other catering premises,7841,Funcraft UK Ltd King George V Dock Woolwich Ma...,London,,...,0.069286,51.501121,None,0001-01-01T00:00:00,0,None,0,0,0,0
6,673ff4ff1a87a14d89721807,878523,0,PI/000029844,Brick Lane Brews,Restaurant/Cafe/Canteen,1,Air side London City Airport,London,,...,0.048006,51.503733,None,0001-01-01T00:00:00,0,None,0,0,0,0
7,673ff4ff1a87a14d8972181f,293772,0,PI/000002585,The Nuance Group (UK) Limited,Retailers - other,4613,Duty Free Shop Passenger Terminal,London City AirportRoyal DocksLondon,,...,0.047966,51.504112,None,0001-01-01T00:00:00,0,None,0,0,0,0
8,673ff4ff1a87a14d89721823,295107,0,PI/000020749,City Bar & Grill,Restaurant/Cafe/Canteen,1,London City Airport (Airside),London,,...,0.0479655750095844,51.5041122436523,None,0001-01-01T00:00:00,0,None,0,0,0,0
9,673ff4ff1a87a14d89721829,294606,0,PI/000016185,WH Smith,Retailers - other,4613,London City Airport,Silvertown London,,...,0.0479655750095844,51.5041122436523,None,0001-01-01T00:00:00,0,None,0,0,0,0


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [23]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

# Extract the 'geocode' field from the document with 'BusinessName' 'Penang Flavours'
lat_long = establishments.find_one({"BusinessName":"Penang Flavours"},{"geocode":1})


degree_search = 0.01
# Assign the lat and long values to variables
latitude = float(lat_long['geocode']['latitude'].to_decimal())
longitude = float(lat_long['geocode']['longitude'].to_decimal())

# Calculate the lat/long search area by +/- the 'degree_search' from the lat/long variables
query = {"geocode.latitude":{'$gte': latitude-degree_search, '$lte': latitude+degree_search},
        "geocode.longitude":{'$gte': longitude-degree_search, '$lte': longitude+degree_search},
        "RatingValue":5}
sort = [("scores.Hygiene",1)]
limit = 5

# Print the results
results = list(establishments.find(query).sort(sort).limit(limit))
for r in results:
    pprint(r)


In [29]:
# Convert result to Pandas DataFrame
near_penang_df = pd.json_normalize(results)

# Export DataFrame to csv file
near_penang_df.to_csv("csv_outputs/near_penang_df.csv", index=False, header=True)

pprint(near_penang_df)

                           _id  count
0                       Thanet   1130
1                    Greenwich    882
2                    Maidstone    713
3                       Newham    711
4                        Swale    686
5                   Chelmsford    680
6                       Medway    672
7                       Bexley    607
8              Southend-On-Sea    586
9                     Tendring    542
10                  Colchester    498
11             Tunbridge Wells    491
12        Folkestone and Hythe    480
13                  Eastbourne    478
14                    Hastings    464
15                     Bromley    460
16                     Ashford    427
17                    Havering    397
18                    Dartford    383
19                   Braintree    382
20                    Basildon    362
21                   Gravesham    339
22       Tonbridge and Malling    325
23                    Thurrock    313
24        Barking and Dagenham    301
25          

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [30]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Pipeline
match = {'$match': {"scores.Hygiene":0}}

group = {'$group': {"_id" : "$LocalAuthorityName",
                   "count": {'$sum':1}}}

sort = {'$sort':{"count":-1}}

pipeline = [match, group, sort]

results = list(artifacts.aggregate(pipeline))

# Print the number of documents in the result
print(f"There are {len(results)} documents in the result.")

# Print the first 10 results
print("")
print("The first 10 documents are:")
print("-"*50)
pprint(results[0:10])
    

There are 55 documents in the result.

The first 10 documents are:
--------------------------------------------------
[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [31]:
# Convert the result to a Pandas DataFrame
agg_df = pd.json_normalize(results)

# Export DataFrame to csv file
agg_df.to_csv("csv_outputs/aggregate_df.csv", index=False, header=True)

# Display the number of rows in the DataFrame
print(f"The number of rows in this DataFrame is: {len(agg_df)}")
print("")

# Display the first 10 rows of the DataFrame
agg_df.head(10)


The number of rows in this DataFrame is: 55



,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
